In [1]:
import cv2
import numpy as np
import mediapipe as mp
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


In [2]:
# b is the midpoint of a and c (e.g. left hip, left elbow and left shoulder)
# our case will be left-hip, left-knee and left-ankle
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

        
    return angle 

In [6]:

# Initialize variables for counter logic
counter = 0
stage = None
confirm_frames = 5
stage_counter = 0
max_angle_per_rep = 0
last_angle = 0

# Initialize variables for optical flow
prev_frame = None
prev_hip = None
prev_knee = None
prev_ankle = None

# for optical flow postprocessing
hip_history = []
knee_history = []
ankle_history = []

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    cap = cv2.VideoCapture("../test/CST_self2.mp4")
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            logging.warning("No frame captured from the video source.")
            break

        # Process the frame
        frame_height, frame_width, _ = frame.shape
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # Extract coordinates multiplied by frame dimensions for optical flow and angle calculation
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x * frame_width, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y * frame_height]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x * frame_width, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y * frame_height]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x * frame_width, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y * frame_height]

            # Calculate angle for sit-stand logic
            angle = calculate_angle(hip, knee, ankle)

            # Optical flow visualization
            if prev_frame is not None:
                for prev_point, curr_point, color in zip([prev_hip, prev_knee, prev_ankle], [hip, knee, ankle], [(0, 0, 255)]*3):
                    prev_x, prev_y = map(int, prev_point)
                    curr_x, curr_y = map(int, curr_point)
                    cv2.circle(image, (curr_x, curr_y), 10, color, -1)
                    cv2.line(image, (prev_x, prev_y), (curr_x, curr_y), (0, 255, 0), 2)

            prev_frame = frame.copy()
            prev_hip, prev_knee, prev_ankle = hip, knee, ankle

            # Display the knee angle beside the knee
            cv2.putText(
                image,
                f"{angle} deg",
                (int(knee[0] + 60), int(knee[1])),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (255, 255, 255),
                2,
                cv2.LINE_AA
            )

            # Counting logic
            if stage is None:
                stage = "up" if angle > 135 else "down"

            if stage == "down" and angle > 135:
                stage_counter += 1
                if stage_counter >= confirm_frames:
                    stage = "up"
                    stage_counter = 0
                    counter += 1
                    logging.info(f"Transitioned to up. Total reps: {counter}")
                    max_angle_per_rep = 0  # Reset max angle for the new repetition

            elif stage == "up" and angle < 90:
                stage_counter += 1
                if stage_counter >= confirm_frames:
                    stage = "down"
                    stage_counter = 0
                    logging.info("Transitioned to down.")

            # Update max angle
            if angle > max_angle_per_rep:
                max_angle_per_rep = angle
            last_angle = angle
            # Render curl counter
            
            # Setup status box
            cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)

            # Rep data
            cv2.putText(
                image,
                "REPS",
                (15, 12),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 0, 0),
                1,
                cv2.LINE_AA,
            )
            cv2.putText(
                image,
                str(counter),
                (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX,
                2,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )

            # Stage data
            cv2.putText(
                image,
                "STAGE",
                (65, 12),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 0, 0),
                1,
                cv2.LINE_AA,
            )
            cv2.putText(
                image,
                stage,
                (60, 60),
                cv2.FONT_HERSHEY_SIMPLEX,
                2,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )

            # Draw landmarks and connections
            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2),
            )
            
        cv2.imshow("Mediapipe Feed", image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1716912158.437115 2366416 gl_context.cc:357] GL version: 2.1 (2.1 ATI-5.2.4), renderer: AMD Radeon Pro 5500M OpenGL Engine
2024-05-29 00:02:44,588 - INFO - Transitioned to up. Total reps: 1
2024-05-29 00:02:53,976 - INFO - Transitioned to down.
